In [ ]:
!pip install moviepy pydub openai-whisper easyocr transformers Pillow exifread


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 20.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 19.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=a9212fade9c3f5694fb91a347b655d88d3a5357a2aa4f19fc94bd390d7829510
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
import moviepy.editor as mp
from pydub import AudioSegment
import whisper
import easyocr
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)

  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)

  elif re.match('(flt)p?( \(default\))?$', token):

  elif re.m

In [4]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
ocr_reader = easyocr.Reader(["en"])
whisper_model = whisper.load_model("base")

# -----------------------------
# 1. Extract video frames
# -----------------------------
def extract_frames(video_path, every_n_seconds=5):
    clip = mp.VideoFileClip(video_path)
    frames = []
    for t in np.arange(0, clip.duration, every_n_seconds):
        frame = clip.get_frame(t)  # returns numpy array
        frames.append(Image.fromarray(frame))
    return frames

# -----------------------------
# 2. Frame analysis (like image analyzer)
# -----------------------------
RISK_WEIGHTS = {
    "face": 35,
    "office/logo": 20,
    "document": 40,
    "group": 15,
    "street sign": 15,
    "ocr_text": 25
}

def analyze_frame(frame):
    # OCR text
    results = ocr_reader.readtext(np.array(frame))
    ocr_text = [res[1] for res in results]

    # CLIP classification
    inputs = clip_processor(
        text=["a person face", "an office with logo", "a document", "a group of people", "a street sign"],
        images=frame,
        return_tensors="pt",
        padding=True
    )
    outputs = clip_model(**inputs)
    probs = outputs.logits_per_image.softmax(dim=1).detach().numpy()[0]
    labels = ["face", "office/logo", "document", "group", "street sign"]
    clip_analysis = {labels[i]: float(probs[i]) for i in range(len(labels))}

    return {"ocr_text": ocr_text, "clip_analysis": clip_analysis}

# -----------------------------
# 3. Extract audio from video
# -----------------------------
def extract_audio_from_video(video_path, audio_path="temp_audio.wav"):
    clip = mp.VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_path)
    return audio_path

def analyze_audio(audio_path):
    # Extract metadata
    audio = AudioSegment.from_file(audio_path)
    metadata = {
        "duration_seconds": len(audio)/1000,
        "channels": audio.channels,
        "frame_rate": audio.frame_rate
    }
    # Transcription
    transcript = whisper_model.transcribe(audio_path)["text"]
    return {"metadata": metadata, "transcript": transcript}

# -----------------------------
# 4. Compute video exposure score
# -----------------------------
def compute_video_exposure_score(frames_analysis, audio_analysis):
    total = 0
    details = []

    # Frame CLIP contributions
    for fa in frames_analysis:
        for label, prob in fa["clip_analysis"].items():
            if prob > 0.5:
                contrib = RISK_WEIGHTS.get(label, 0) * prob
                total += contrib
                details.append((label, round(contrib,2)))

        # OCR text
        if fa["ocr_text"]:
            total += RISK_WEIGHTS["ocr_text"]
            details.append(("ocr_text", RISK_WEIGHTS["ocr_text"]))

    # Audio keyword detection
    SENSITIVE_KEYWORDS = ["password", "ssn", "credit card", "secret", "confidential"]
    keyword_count = sum(audio_analysis["transcript"].lower().count(kw) for kw in SENSITIVE_KEYWORDS)
    if keyword_count:
        contrib = min(50, keyword_count * 10)
        total += contrib
        details.append(("audio_sensitive_keywords", contrib))

    total = min(100, round(total,2))
    return {"exposure_score": total, "details": details}

# -----------------------------
# 5. Full video analysis
# -----------------------------
def analyze_video(video_path):
    print("Extracting frames...")
    frames = extract_frames(video_path, every_n_seconds=5)

    print("Analyzing frames...")
    frames_analysis = [analyze_frame(f) for f in frames]

    print("Extracting audio...")
    audio_path = extract_audio_from_video(video_path)

    print("Analyzing audio...")
    audio_analysis = analyze_audio(audio_path)

    print("Computing exposure score...")
    score = compute_video_exposure_score(frames_analysis, audio_analysis)

    return {"frames_analysis": frames_analysis, "audio_analysis": audio_analysis, "score": score}

# -----------------------------
# 6. Colab file upload & run
# -----------------------------
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\nAnalyzing {filename}...\n")
    result = analyze_video(filename)
    print(f"\nOverall Exposure Score: {result['score']['exposure_score']}/100")
    print("Details:")
    for d in result["score"]["details"]:
        print(f"- {d[0]} → +{d[1]}")
    print(f"\nAudio Transcript:\n{result['audio_analysis']['transcript']}")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Saving 6247922-uhd_2160_3840_24fps.mp4 to 6247922-uhd_2160_3840_24fps.mp4

Analyzing 6247922-uhd_2160_3840_24fps.mp4...

Extracting frames...
Analyzing frames...
Extracting audio...
MoviePy - Writing audio in temp_audio.wav


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



MoviePy - Done.
Analyzing audio...
Computing exposure score...

Overall Exposure Score: 35.35/100
Details:
- group → +9.4
- face → +25.95

Audio Transcript:

